#Importing data from kaggle

In [1]:
!kaggle datasets download -d faizalkarim/flood-area-segmentation

Dataset URL: https://www.kaggle.com/datasets/faizalkarim/flood-area-segmentation
License(s): CC0-1.0
 99% 106M/107M [00:05<00:00, 24.1MB/s]
100% 107M/107M [00:05<00:00, 19.0MB/s]


Extracting the .zip file


In [2]:
import zipfile

# Create a ZipFile object, providing the path to your ZIP file
with zipfile.ZipFile('/content/flood-area-segmentation.zip', 'r') as zip_ref:
    # Use the extractall method of the ZipFile object to extract all contents
    zip_ref.extractall('/content/') # You can specify a destination directory here

#Imorting nessecary libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from sklearn.model_selection import train_test_split


In [4]:

def load_image_and_mask(image_dir, mask_dir, target_size):
    images = []
    masks = []

    # Sort the filenames to ensure corresponding images and masks are loaded together
    image_filenames = sorted(os.listdir(image_dir))
    mask_filenames = sorted(os.listdir(mask_dir))

    # Ensure that images and masks have the same filenames (ignoring extensions)
    for img_name, mask_name in zip(image_filenames, mask_filenames):
        if os.path.splitext(img_name)[0] == os.path.splitext(mask_name)[0]:
            # Load and process the image
            img_path = os.path.join(image_dir, img_name)
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img) / 255.0  # Normalize image to 0-1
            images.append(img_array)

            # Load and process the mask
            mask_path = os.path.join(mask_dir, mask_name)
            mask = load_img(mask_path, target_size=target_size, color_mode="grayscale")
            mask_array = img_to_array(mask)
            mask_array = np.where(mask_array > 0, 1, 0)  # Convert mask to binary (0 or 1)
            masks.append(mask_array)

    # Convert lists to numpy arrays
    images = np.array(images)
    masks = np.array(masks)

    return images, masks


In [5]:
# Directories for images and masks
image_dir = r'/content/Image'
mask_dir = r'/content/Mask'


In [6]:

# Load image and mask arrays
X, Y = load_image_and_mask(image_dir, mask_dir, target_size=(256, 256))


##Creating CNN Model

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, Model


In [12]:

# Input layer
inputs = layers.Input(shape=(256, 256, 3))

# First Conv Block
conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
pool1 = layers.MaxPooling2D((2, 2))(conv1)

# Second Conv Block
conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
pool2 = layers.MaxPooling2D((2, 2))(conv2)

# Third Conv Block
conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
pool3 = layers.MaxPooling2D((2, 2))(conv3)

# Upsampling layers (to restore the size back to 256x256)
up1 = layers.UpSampling2D((2, 2))(pool3)
up_conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up1)

up2 = layers.UpSampling2D((2, 2))(up_conv1)
up_conv2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up2)

up3 = layers.UpSampling2D((2, 2))(up_conv2)
up_conv3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up3)

# Output layer (256, 256, 1) for the mask
outputs = layers.Conv2D(1, (1, 1), activation='sigmoid', padding='same')(up_conv3)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 256, 256, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 128, 128, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 64, 64, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d (UpSampling2D)         │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 64, 64, 64)          │          73,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_1 (UpSampling2D)       │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 128, 128, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_2 (UpSampling2D)       │ (None, 256, 256, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 256, 256, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 256, 256, 1)         │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 194,785 (760.88 KB)

 Trainable params: 194,785 (760.88 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the model using the images (X) and masks (Y)
 model.fit(X, Y, epochs=10, batch_size=32)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 149s 14s/step - accuracy: 0.5893 - loss: 0.6617
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 191s 13s/step - accuracy: 0.7541 - loss: 0.5388
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 142s 13s/step - accuracy: 0.6981 - loss: 0.5808
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 144s 14s/step - accuracy: 0.7661 - loss: 0.5126
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 131s 13s/step - accuracy: 0.7871 - loss: 0.4788
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 142s 13s/step - accuracy: 0.7891 - loss: 0.4558
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 142s 13s/step - accuracy: 0.7812 - loss: 0.4932
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 150s 14s/step - accuracy: 0.7818 - loss: 0.4846
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 133s 13s/step - accuracy: 0.8010 - loss: 0.4500
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 130s 13s/step - accuracy: 0.8123 - loss: 0.4228


##Save simple CNN model

In [17]:

model.save('model_name.h5')


#Testing Unet

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# Define the U-Net architecture
inputs = layers.Input(shape=(256, 256, 3))

# Encoder (Downsampling)
conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
pool1 = layers.MaxPooling2D((2, 2))(conv1)

conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
pool2 = layers.MaxPooling2D((2, 2))(conv2)

conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
pool3 = layers.MaxPooling2D((2, 2))(conv3)

conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
conv4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
pool4 = layers.MaxPooling2D((2, 2))(conv4)

# Bottleneck
conv5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
conv5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)

# Decoder (Upsampling)
up6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
concat6 = layers.concatenate([up6, conv4])
conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(concat6)
conv6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

up7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
concat7 = layers.concatenate([up7, conv3])
conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(concat7)
conv7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

up8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
concat8 = layers.concatenate([up8, conv2])
conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(concat8)
conv8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

up9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
concat9 = layers.concatenate([up9, conv1])
conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(concat9)
conv9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

# Output Layer (256, 256, 1 for mask)
outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(conv9)

# Create the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary (optional)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 64)   │          1,792 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 256, 256, 64)   │         36,928 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 64)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 128, 128, 128)  │        147,584 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 128)    │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 32, 32, 256)    │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 32, 32, 512)    │      1,180,160 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 32, 32, 512)    │      2,359,808 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 16, 16, 512)    │              0 │ conv2d_7[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 16, 16, 1024)   │      4,719,616 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 16, 16, 1024)   │      9,438,208 │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_transpose          │ (None, 32, 32, 512)    │      2,097,664 │ conv2d_9[0][0]         │
│ (Conv2DTranspose)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 32, 32, 1024)   │              0 │ conv2d_transpose[0][0… │
│                      

 Total params: 31,031,745 (118.38 MB)

 Trainable params: 31,031,745 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Train the model using the images (X) and masks (Y)
model.fit(X, Y, epochs=10, batch_size=32)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 167s 4s/step - accuracy: 0.5729 - loss: 0.6790
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 56s 1s/step - accuracy: 0.5921 - loss: 0.9978
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.6227 - loss: 0.7063
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.6763 - loss: 0.6829
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.6733 - loss: 0.6759
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.6408 - loss: 0.6485
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.6662 - loss: 0.7127
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.7257 - loss: 0.5557
Epoch 9/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.7613 - loss: 0.4992
Epoch 10/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.7415 - loss: 0.5240


In [11]:
# Train the model using the images (X) and masks (Y)
model.fit(X, Y, epochs=50, batch_size=16)

Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 13s 680ms/step - accuracy: 0.8297 - loss: 0.3875
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 13s 695ms/step - accuracy: 0.8349 - loss: 0.3779
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 13s 699ms/step - accuracy: 0.8342 - loss: 0.3831
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 687ms/step - accuracy: 0.8182 - loss: 0.4078
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 13s 684ms/step - accuracy: 0.8303 - loss: 0.3840
Epoch 6/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 683ms/step - accuracy: 0.8310 - loss: 0.3763
Epoch 7/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 691ms/step - accuracy: 0.8267 - loss: 0.3828
Epoch 8/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 688ms/step - accuracy: 0.8379 - loss: 0.3714
Epoch 9/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 13s 689ms/step - accuracy: 0.8300 - loss: 0.3845
Epoch 10/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 680ms/step - accuracy: 0.8320 - loss: 0.3786
Epoch 11/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 687ms/step - accuracy: 0.8241 - loss: 0.3971
Epoch 12/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 21

In [10]:

model.save('model_unet.h5')
